![QA With History Aware](https://python.langchain.com/v0.2/assets/images/conversational_retrieval_chain-5c7a96abe29e582bc575a0a0d63f86b0.png)

In [28]:
! pip uninstall --quiet transformers -y
! pip uninstall --quiet tokenizers -y
! pip uninstall --quiet langchain-huggingface -y
! pip uninstall --quiet langchain_cohere -y
! pip uninstall --quiet Cohere -y


In [4]:
! pip install --quiet langchain

In [5]:
! pip install --quiet bs4
! pip install --quiet pymupdf
! pip install -q langchain-pinecone

In [42]:
! pip install -q sentence_transformers

In [7]:
! pip install --quiet langchain_community

In [29]:
!pip install -q tokenizers==0.19.1

In [30]:
!pip install -q cohere

In [31]:
!pip install -q langchain_cohere

In [32]:
# setup
import os
from google.colab import userdata
import logging


In [33]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # Check if COHERE_API_KEY exists in environment variables, if not, try to fetch it from userdata
    if "COHERE_API_KEY" not in os.environ:
        cohere_api_key = userdata.get('COHERE_API_KEY')
        if cohere_api_key:
            os.environ["COHERE_API_KEY"] = cohere_api_key
            logging.info("COHERE_API_KEY set from userdata.")
        else:
            raise ValueError("COHERE_API_KEY not found in environment variables or userdata.")

    # Similarly for LANGCHAIN_API_KEY
    if "LANGCHAIN_API_KEY" not in os.environ:
        langchain_api_key = userdata.get('LANGCHAIN_API_KEY')
        if langchain_api_key:
            os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
            logging.info("LANGCHAIN_API_KEY set from userdata.")
        else:
            raise ValueError("LANGCHAIN_API_KEY not found in environment variables or userdata.")

    # Similarly for PINECONE_API_KEY
    if "PINECONE_API_KEY" not in os.environ:
        pinecode_api_key = userdata.get('PINECONE_API_KEY')
        if pinecode_api_key:
            os.environ["PINECONE_API_KEY"] = pinecode_api_key
            logging.info("PINECONE_API_KEY set from userdata.")
        else:
            raise ValueError("PINECONE_API_KEY not found in environment variables or userdata.")

    # Set other environment variables
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "HarryPotter-book-Chatbot"
    logging.info("Other environment variables set.")

except (KeyError, ValueError) as e:
    logging.error(f"An error occurred: {e}")

In [34]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the PDF file
file_path = "/content/harrypotter_all_books.pdf"
loader = PyMuPDFLoader(file_path)
try:
    pages = loader.load()
    logging.info("Successfully loaded {len(pages)} pages")
except Exception as e:
    logging.error(f"Error loading PDF with PyMuPDFLoader: {e}")


In [35]:
len(pages)

3623

In [36]:
pages[40].page_content

'“I want to read it,” said Harry furiously, “as it’s mine.”\n“Get out, both of you,” croaked Uncle Vernon, stuffing the letter back\ninside its envelope.\nHarry didn’t move.\n“I WANT MY LETTER!” he shouted.\n“Let me see it!” demanded Dudley.\n“OUT!” roared Uncle Vernon, and he took both Harry and Dudley by the\nscruffs of their necks and threw them into the hall, slamming the kitchen door\nbehind them. Harry and Dudley promptly had a furious but silent fight over\nwho would listen at the keyhole; Dudley won, so Harry, his glasses dangling\nfrom one ear, lay flat on his stomach to listen at the crack between door and\nfloor.\n“Vernon,” Aunt Petunia was saying in a quivering voice, “look at the\naddress — how could they possibly know where he sleeps? You don’t think\nthey’re watching the house?”\n“Watching — spying — might be following us,” muttered Uncle Vernon\nwildly.\n“But what should we do, Vernon? Should we write back? Tell them we\ndon’t want —”\nHarry could see Uncle Vernon’s shi

In [37]:

# Split the pages into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
try:
    split_pages = text_splitter.split_documents(documents=pages)
    logging.info("Pages split into {len(split_pages)} chunks.")
except Exception as e:
    logging.error(f"Error splitting pages: {e}")


In [38]:
len(split_pages)

8891

In [39]:
split_pages[40].page_content

'N\n \nCHAPTER  TWO\nTHE VANISHING GLASS\nearly ten years had passed since the Dursleys had woken up to find\ntheir nephew on the front step, but Privet Drive had hardly changed at\nall. The sun rose on the same tidy front gardens and lit up the brass number\nfour on the Dursleys’ front door; it crept into their living room, which was\nalmost exactly the same as it had been on the night when Mr. Dursley had\nseen that fateful news report about the owls. Only the photographs on the\nmantelpiece really showed how much time had passed. Ten years ago, there\nhad been lots of pictures of what looked like a large pink beach ball wearing\ndifferent-colored bonnets — but Dudley Dursley was no longer a baby, and\nnow the photographs showed a large blond boy riding his first bicycle, on a\ncarousel at the fair, playing a computer game with his father, being hugged\nand kissed by his mother. The room held no sign at all that another boy lived\nin the house, too.'

In [43]:
# # Create Cohere embeddings

# from langchain_cohere import CohereEmbeddings

# try:
#     embeddings = CohereEmbeddings()
#     logging.info("Cohere embeddings created.")

# except Exception as e:
#     logging.error(f"Error creating Cohere embeddings: {e}")


from langchain.embeddings import SentenceTransformerEmbeddings

try:
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    logging.info("SentenceTransformer embeddings created.")

except Exception as e:
    logging.error(f"Error creating SentenceTransformer embeddings: {e}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Pinecone vector database

In [50]:

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))


In [51]:
import time

index_name = "harrypotter-index"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [52]:
from langchain.vectorstores import Pinecone

# Embed and store the chunks in Pinecone
try:
    vectorstore = Pinecone.from_texts([doc.page_content for doc in split_pages], embeddings, index_name=index_name)
    logging.info(f"Successfully embedded chunk in pinecone index_name: {index_name} ")
except Exception as e:
    logging.error(f"Embedding failed for chunk: {e}", exc_info=True)


In [ ]:
# from langchain.vectorstores import FAISS

# try:
#     # Load the vector store from a file if it exists
#     # Assuming 'faiss_index' is the directory where the index is saved
#     vectorstore = FAISS.load_local("faiss_index", embeddings)
#     logging.info("FAISS vector store loaded from file.")
# except FileNotFoundError:
#     # If the vector store file is not found, create a new one
#     vectorstore = FAISS.from_documents(documents=split_pages, embedding=embeddings)
#     logging.info("FAISS vector store created.")
#     # Save the vector store to a file
#     vectorstore.save_local("faiss_index")
#     logging.info("FAISS vector store saved to file.")

# # Now you can serialize the vectorstore object if needed
# vectorstore = vectorstore.serialize_to_bytes()

In [55]:
# retriever
try:
    retriever = vectorstore.as_retriever()
    logging.info("Retriever created.")
except Exception as e:
    logging.error(f"Error creating retriever: {e}")


In [56]:
# model
from langchain_cohere import ChatCohere

chat_model = ChatCohere(model="command-nightly", temperature=0.3)

In [57]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)


In [60]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

try:
    history_aware_retriever = create_history_aware_retriever(llm=chat_model, retriever=retriever, prompt=contextualize_q_prompt)
    logging.info("History-aware retriever created.")
except Exception as e:
    logging.error(f"Error creating history-aware retriever: {e}")


In [61]:
### Answer question ###

from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

try:
    question_answer_chain = create_stuff_documents_chain(llm=chat_model, prompt=qa_prompt)
    logging.info("Question-answering chain created.")
except Exception as e:
    logging.error(f"Error creating question-answering chain: {e}")

try:
    rag_chain = create_retrieval_chain(retriever=history_aware_retriever, combine_docs_chain=question_answer_chain)
    logging.info("RAG chain created.")
except Exception as e:
    logging.error(f"Error creating RAG chain: {e}")

In [62]:
### Statefully manage chat history ###
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [64]:
conversational_rag_chain.invoke(
    {"input": "describe harry potter character from the whole book?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Harry Potter is brave, loyal, and determined. He is a natural leader with a strong sense of justice. He is also humble, despite his fame and popularity.'